In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install pandas

In [9]:
import os
import librosa
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score

In [3]:
# for 3 second sampels
def load_and_concatenate_npy_files(file_paths):
    """
    Load multiple np.array files from Google Drive and concatenate them into a single array.

    Parameters:
    file_paths (list of str): List of paths to the np.array files on Google Drive.

    Returns:
    np.array: Concatenated array.
    """
    # Initialize an empty list to hold arrays
    arrays = []

    # Iterate through the file paths, load each file, and append to the list
    for file_path in file_paths:
        # Load the np.array file
        array = np.load(file_path)
        # Append the loaded array to the list
        arrays.append(array)

    # Concatenate all arrays into a single array
    concatenated_array = np.concatenate(arrays, axis=0)

    return concatenated_array

# Example usage:
file_paths = [
    '/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings_1.npy',
    '/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings_2.npy',
    '/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings_3.npy',
    '/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_train_embeddings_4.npy']


pod_train_embeddings =load_and_concatenate_npy_files(file_paths)
#[:852, :]
ad_train_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_ad_train_embeddings.npy")

#pod_val_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_val_embeddings.npy")
#ad_val_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_ad_val_embeddings.npy")

#pod_test_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_pod_test_embeddings.npy")
#ad_test_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_3sec/OpenL3_3sec_ad_test_embeddings.npy")

live_radio_embeddings = np.load("/content/drive/MyDrive/AD-Blocker Project/embeddings/OpenL3_live_radio_3_sec/OpenL3_live_radio_3_sec_embeddings.npy")

print("pod train embeddings shape:", pod_train_embeddings.shape)
print("ad train embeddings shape:", ad_train_embeddings.shape)
print("live radio embeddings shape:", live_radio_embeddings.shape)

#print("pod_val", pod_val_embeddings.shape)
#print("ad_val", ad_val_embeddings.shape)

#print("pod_test", pod_test_embeddings.shape)
#print("ad_test", ad_test_embeddings.shape)

pod train embeddings shape: (5260, 13312)
ad train embeddings shape: (852, 13312)
live radio embeddings shape: (709, 13312)


In [ ]:
def convert_to_2d_array(three_d_array):
    # Get the dimensions of the input array
    depth, rows, cols = three_d_array.shape

    # Reshape each 2D array to 1D and concatenate them
    flattened_arrays = [matrix.flatten() for matrix in three_d_array]
    two_d_array = np.vstack(flattened_arrays)

    return two_d_array

In [4]:
train_embeddings = np.concatenate((pod_train_embeddings, ad_train_embeddings))
test_embeddings = live_radio_embeddings

In [5]:
def concatenate_zeros_and_ones(podcast_length, commercials_length):
    # Create array of zeros with size of podcast array
    zeros_array = np.zeros(podcast_length, dtype=int)

    # Create array of ones with size of commercials array
    ones_array = np.ones(commercials_length, dtype=int)

    # Concatenate arrays
    concatenated_array = np.concatenate((zeros_array, ones_array))

    return concatenated_array


podcast_length = pod_train_embeddings.shape[0]
ads_length = ad_train_embeddings.shape[0]

In [6]:
train_labels = concatenate_zeros_and_ones(pod_train_embeddings.shape[0], ad_train_embeddings.shape[0])
print(train_labels.shape)

(6112,)


In [10]:
live_radio_excel_path = "/content/drive/MyDrive/AD-Blocker Project/real_time_audio/audio_segments.xlsx"

df = pd.read_excel(live_radio_excel_path)
labels = df['Label'].to_numpy()

# Step 6: Take all elements except for the last item
labels_except_last = labels[:-1]

# Display the array
print(labels_except_last)

print(labels_except_last.shape)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

In [11]:
# Define sample weights
# Assuming you have a binary classification problem with labels 0 (podcast) and 1 (commercial)
# Assign weight 1 to podcast samples and weight 6 to commercial samples
sample_weights = {0: 1, 1: 1}

def train_svm_model(embeddings, labels, weights=None):
    """
    Train an SVM model using the provided embeddings and labels.

    Parameters:
    - embeddings: List of embeddings (list of arrays).
    - labels: List of corresponding labels.
    - weights: Dictionary specifying the class weights. Default is None.

    Returns:
    - model: Trained SVM model.
    """
    # Initialize SVM model with linear kernel and class weights
    model = SVC(kernel='linear', class_weight=weights)
    # Train the model
    model.fit(embeddings, labels)
    return model

In [16]:
# Train the SVM model with sample weights
model = train_svm_model(train_embeddings, train_labels)

In [17]:
def test_model(model, test_embeddings, test_labels):
    """
    Test the trained model on a separate test dataset.

    Parameters:
    - model: Trained machine learning model.
    - test_embeddings: Embeddings of the test dataset.
    - test_labels: Labels of the test dataset.

    Returns:
    - test_accuracy: Accuracy of the model on the test dataset.
    - predicted_labels: Predicted labels for the test dataset.
    - actual_labels: Actual labels of the test dataset.
    """
    # Make predictions on the test dataset
    predicted_labels = model.predict(test_embeddings)

    # Calculate test accuracy
    test_accuracy = accuracy_score(test_labels, predicted_labels)

    # Print test accuracy
    print("Test Accuracy:", test_accuracy)

    # Print predicted and actual labels for inspection
    print("Predicted labels:")
    print(predicted_labels)
    print("Actual labels:")
    print(test_labels)

    # Return test accuracy and predicted/actual labels
    return test_accuracy, predicted_labels, test_labels

In [18]:
# Call the function by passing the required arguments
test_accuracy, test_predicted_labels, test_actual_labels = test_model(model, live_radio_embeddings, labels_except_last)

# Now you can use the returned values as needed
print("Test Accuracy:", test_accuracy)

Test Accuracy: 0.6727785613540197
Predicted labels:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [19]:
def calculate_confusion_matrix(actual_labels, predicted_labels):
    """
    Calculate the confusion matrix based on actual and predicted labels.

    Parameters:
    actual_labels (list): List of actual labels (-1 for ads, 1 for podcasts).
    predicted_labels (list): List of predicted labels (-1 for ads, 1 for podcasts).

    Returns:
    tuple: A tuple containing true positives, false positives, true negatives, and false negatives.
    """
    true_positives = 0
    false_positives = 0
    true_negatives = 0
    false_negatives = 0

    for actual, predicted in zip(actual_labels, predicted_labels):
        if actual == 0 and predicted == 0:  # true_negatives
            true_negatives += 1
        elif actual == 1 and predicted == 0:  # false_negatives
            false_negatives += 1
        elif actual == 1 and predicted == 1:  # true_positives
            true_positives += 1
        elif actual == 0 and predicted == 1:  # false_positives
            false_positives += 1

    return true_positives, false_positives, true_negatives, false_negatives

In [24]:
# Call the function by passing the actual labels and predicted labels
true_positives, false_positives, true_negatives, false_negatives = calculate_confusion_matrix(labels_except_last, test_predicted_labels)

# Print the results
print("True Positives:", true_positives)
print("False Positives:", false_positives)
print("True Negatives:", true_negatives)
print("False Negatives:", false_negatives)

# Calculate evaluation values
#precision = true_positives / (true_positives + false_positives)
recall = true_positives / (true_positives + false_negatives)
specificity = true_negatives / (true_negatives + false_positives)
#f1_score = 2 * (precision * recall) / (precision + recall)

# Print the evaluation values
#print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity)
#print("F1 Score:", f1_score)

True Positives: 0
False Positives: 0
True Negatives: 477
False Negatives: 232
Recall: 0.0
Specificity: 1.0
